In [163]:
# Importing modules...
import numpy as np
import cv2
import time

import sys
sys.path.append('../..') # Go back to base directory

from modules.vision.camera import Camera
from modules.vision.epipolar_geometry import build_essential_matrix, build_fundamental_matrix, order_centroids
from modules.vision.blob_detection import detect_blobs

from coppeliasim_zmqremoteapi_client import RemoteAPIClient

# Init client 
client = RemoteAPIClient()    # Client object 
sim = client.getObject('sim') # Simulation object

In [164]:
n_cameras = 8
camera = [] # Camera objects list

for ID in range(n_cameras):
    # Get the vision sensor handle
    vision_sensor_handle = sim.getObject(f'/VisionSensor[{ID}]')

    # Get Coppelia's 3x4 object matrix
    object_matrix = np.array(sim.getObjectMatrix(vision_sensor_handle)).reshape((3,4))

    # X and Y axis of Coppelia's Vision Sensor are inverted
    object_matrix[:,0] *= -1 # Invert x vector column
    object_matrix[:,1] *= -1 # Invert y vector column

    camera.append(Camera(# Simulation handling
                          vision_sensor_handle=vision_sensor_handle,
   
                          # Intrinsic Parameters
                          resolution=(720,720), 
                          fov_degrees=60.0,     
          
                          # Extrinsic Parameters
                          object_matrix=object_matrix,
          
                          # Rational Lens Distortion Model
                          # distortion_model='rational',
                          # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]),

                          # Fisheye Lens Distortion Model
                          distortion_model='fisheye',
                          distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
          
                          # Image Noise Model
                          snr_dB=13))

In [165]:
fundamental_matrix = np.array(np.zeros((n_cameras, n_cameras, 3, 3)))

for reference in range(n_cameras):
    for auxiliary in range(n_cameras):
        if reference == auxiliary:
            continue

        E = build_essential_matrix(camera[reference].extrinsic_matrix, camera[auxiliary].extrinsic_matrix)

        F = build_fundamental_matrix(camera[reference].intrinsic_matrix, camera[auxiliary].intrinsic_matrix, E)

        fundamental_matrix[reference][auxiliary] = F

In [166]:
# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default.
# Setting the idle loop to run at full speed for this program
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)   
sim.setInt32Param(sim.intparam_idle_fps, 0)

pair = (0, 1)
reference, auxiliary = pair

markers_positions = {} # Markers positions will be stored here

simulation_time = 5 # In seconds
images = {}

# Simulation begins here
sim.startSimulation()

timer_start = time.time()

while (t := sim.getSimulationTime()) < simulation_time:
    # Get and save images
    sync_images = [camera[ID].get_coppelia_image(sim.getVisionSensorCharImage) for ID in range(n_cameras)]
    images[t] = sync_images    

    # Detect blobs on the distorted image
    distorted_blobs = [detect_blobs(image) for image in sync_images]

    # If no blobs were detected then images are invalid
    if any(blob is None for blob in distorted_blobs):
        continue # Jump to next iteration

    # Undistort blobs
    if camera[0].distortion_model == 'rational':
        undistorted_blobs = [cv2.undistortPoints(src=distorted_blobs[ID].T.reshape(1, -1, 2).astype(np.float32), 
                                                cameraMatrix=camera[ID].intrinsic_matrix, 
                                                distCoeffs=camera[ID].distortion_coefficients,
                                                P=camera[ID].intrinsic_matrix).reshape(-1,2).T for ID in range(n_cameras)]
        
    elif camera[0].distortion_model == 'fisheye':
        undistorted_blobs = [cv2.fisheye.undistortPoints(distorted=distorted_blobs[ID].T.reshape(1, -1, 2).astype(np.float32), 
                                                         K=camera[ID].intrinsic_matrix, 
                                                         D=camera[ID].distortion_coefficients,
                                                         P=camera[ID].intrinsic_matrix).reshape(-1,2).T for ID in range(n_cameras)]

    # Order blobs
    epilines_auxiliary = cv2.computeCorrespondEpilines(points=undistorted_blobs[reference].T, 
                                                       whichImage=1, 
                                                       F=fundamental_matrix[reference][auxiliary]).reshape(-1,3)
    
    undistorted_blobs[auxiliary] = order_centroids(undistorted_blobs[auxiliary], epilines_auxiliary)

    # Triangulate markers
    triangulated_positions = cv2.triangulatePoints(camera[reference].projection_matrix.astype(float), 
                                                   camera[auxiliary].projection_matrix.astype(float), 
                                                   undistorted_blobs[reference].astype(float), 
                                                   undistorted_blobs[auxiliary].astype(float))
    
    # Normalize homogeneous coordinates and save
    markers_positions[t] = (triangulated_positions / triangulated_positions[-1])[:-1, :]

timer_end = time.time()

# Closing all open windows 
cv2.destroyAllWindows()

# Restore the original idle loop frequency:
sim.setInt32Param(sim.intparam_idle_fps, defaultIdleFps)

# Simulation ends here
sim.stopSimulation()

print(f'Real Time Factor: {simulation_time / (timer_end - timer_start)}')

Real Time Factor: 0.23198986460296125


In [167]:
# Calculate Accuracy and Precision (only works for one marker)
all_marker_positions = None
for frame, pts in enumerate(markers_positions.keys()):
    if all_marker_positions is None:
        all_marker_positions = markers_positions[pts]
    else:
        all_marker_positions = np.hstack((all_marker_positions, markers_positions[pts]))

ground_truth_position = np.array(sim.getObjectPosition(sim.getObject('/Marker')))
mean_position = np.mean(all_marker_positions, axis=1)
std_dev_position = np.std(all_marker_positions, axis=1)

print('Ground Truth Position:', ground_truth_position)
print('Mean Position:', mean_position)
print(f'Triangulation Error: {np.linalg.norm(mean_position - ground_truth_position) * 1e3 :.2f} mm')
print(f'Standard Deviation: {np.linalg.norm(std_dev_position) * 1e3 :.2f} mm')

# Playback a camera image feed in real time
ID = 0
timestep = int(1000 * simulation_time/len(images.keys()))
for frame, pts in enumerate(images.keys()):
    cv2.imshow(f'Camera {ID}', images[pts][ID])
    cv2.waitKey(timestep)

# Closing all open windows 
cv2.destroyAllWindows()

Ground Truth Position: [-1.175 -0.975  0.1  ]
Mean Position: [-1.17517748 -0.98312548  0.09942744]
Triangulation Error: 8.15 mm
Standard Deviation: 2.30 mm
